# Create segmentation masks for all flood-related disaster images in xBD (only post-disaster)

In [4]:
%%bash
cd ..
. scripts/config/flood.sh
echo 'bucket_path: '${bucket_path}
echo 'seg_model_path: '${seg_model_path} 
echo 'seg_model_name: '${seg_model_name} 

echo 'root_dir: '${root_dir} 
echo 'dataroot: '${dataroot}
echo 'model_name: '${model_name}
echo 'ckpt_dir: '${ckpt_dir}
echo 'temp_dir: '${temp_dir}
echo 'results_dir: '${results_dir}

bucket_path: eie_floods/naip/xbd/images/
seg_model_path: pretrained/pix2pix_seg/flood/scratch_1024_plus/
seg_model_name: latest_net_G.pth
root_dir: /home/jupyter/eie_vision/
dataroot: data/eie_floods/naip/xbd/images
model_name: conditional_binary_scratch_naip/
ckpt_dir: temp/checkpoint/Pix2pixHD/
temp_dir: temp/Pix2pixHD/
results_dir: results/Pix2pixHD/


## Load dataset into instance

In [17]:
%%bash
# set -xe
# Load xBD full data into instance to create segmentation masks
cd ..; . scripts/config/flood.sh 
bucket_path='eie_xview_processed/images/' # 'eie-flood-imagery-prediction/data/xBD_full/'# 
split_src=('Train_B/' 'Test_B/' 'Hold_B/' 'Tier3_B/')
split_dst=('train_B/' 'test_B/' 'hold_B/' 'tier3_B/')
for ((i=0;i<${#split_src[@]};++i)); do 
    mkdir -p ${root_dir}data/${bucket_path}${split_dst[i]} 
    gsutil -m cp -r gs://${bucket_path}${split_src[i]}* ${root_dir}data/${bucket_path}${split_dst[i]} 
done

Copying gs://eie_xview_processed/images/Train_B/hurricane-florence_00000000_post_disaster.png...
Copying gs://eie_xview_processed/images/Train_B/hurricane-florence_00000001_post_disaster.png...
Copying gs://eie_xview_processed/images/Train_B/hurricane-florence_00000002_post_disaster.png...
Copying gs://eie_xview_processed/images/Train_B/hurricane-florence_00000003_post_disaster.png...
Copying gs://eie_xview_processed/images/Train_B/hurricane-florence_00000008_post_disaster.png...
Copying gs://eie_xview_processed/images/Train_B/hurricane-florence_00000012_post_disaster.png...
Copying gs://eie_xview_processed/images/Train_B/hurricane-florence_00000015_post_disaster.png...
Copying gs://eie_xview_processed/images/Train_B/hurricane-florence_00000016_post_disaster.png...
Copying gs://eie_xview_processed/images/Train_B/hurricane-florence_00000018_post_disaster.png...
Copying gs://eie_xview_processed/images/Train_B/hurricane-florence_00000020_post_disaster.png...
Copying gs://eie_xview_process

In [1]:
%%bash

# root_dir='/home/jupyter/eie_vision/'
cd ..
. scripts/config/flood.sh 

bucket_path='eie_xview_processed/images/'
split_src=('test_B/' 'hold_B/' 'tier3_B/' 'train_B/')
split_dst=('test_mask/' 'hold_mask/' 'tier3_mask/' 'train_mask/') # 

for ((i=0;i<${#split_src[@]};++i)); do 
    src_path=${root_dir}data/${bucket_path}${split_src[i]}
    dst_path=${root_dir}data/${bucket_path}${split_dst[i]}
    
    # Delete pre-existing segmentation masks
    rm -r ${dst_path} | true

    # Create segmentation masks
    python src/models/Pix2pix-CycleGAN/test.py --phase to_segment \
    --dataroot ${src_path} --direction AtoB \
    --results_dir ${dst_path} \
    --dataset_mode single --model pix2pix \
    --checkpoints_dir ${seg_model_path} \
    --name . --num_test 10000 --no_flip --gpu_ids 0,1,2,3,4,5,6,7 --max_dataset_size 10000 \
    --batch_size 8  --load_size 1024 --crop_size 1024 || true
    # --num_test 775
    # TODO: eliminate FileNotFoundError: of .../test_mask./loses.csv'
    
    mv ${dst_path}to_segment_latest/images/*_fake_B.png ${dst_path}
    rm -r ${dst_path}to_segment_latest | true
    rename 's/_fake_B.png/.png/' ${dst_path}* #${src_file}
done


KeyboardInterrupt



In [19]:
%%bash 

# Import paths (has to be called from root)
cd ..
. scripts/config/flood.sh
split_dst=('test_mask/' 'hold_mask/' 'tier3_mask/' 'train_mask/')
bucket_path='eie_xview_processed/images/'
dst_bucket_path='eie-flood-imagery-prediction/data/xBD_flood/'

for ((i=0;i<${#split_dst[@]};++i)); do 
    dst_path=${root_dir}data/${bucket_path}
    gsutil -m cp -r ${dst_path}${split_dst[i]}*.png gs://${dst_bucket_path}${split_dst[i]}
done

Copying file:///home/jupyter/eie_vision/data/eie_xview_processed/images/test_mask/hurricane-florence_00000004_post_disaster.png [Content-Type=image/png]...
Copying file:///home/jupyter/eie_vision/data/eie_xview_processed/images/test_mask/hurricane-florence_00000005_post_disaster.png [Content-Type=image/png]...
Copying file:///home/jupyter/eie_vision/data/eie_xview_processed/images/test_mask/hurricane-florence_00000007_post_disaster.png [Content-Type=image/png]...
Copying file:///home/jupyter/eie_vision/data/eie_xview_processed/images/test_mask/hurricane-florence_00000013_post_disaster.png [Content-Type=image/png]...
Copying file:///home/jupyter/eie_vision/data/eie_xview_processed/images/test_mask/hurricane-florence_00000019_post_disaster.png [Content-Type=image/png]...
Copying file:///home/jupyter/eie_vision/data/eie_xview_processed/images/test_mask/hurricane-florence_00000029_post_disaster.png [Content-Type=image/png]...
Copying file:///home/jupyter/eie_vision/data/eie_xview_processed

### Copy all pre-flood images from eie_xview_processed into xbd_flood

In [ ]:
%%bash
# set -xe
cd ..; . scripts/config/flood.sh 
bucket_path='eie_xview_processed/images/' # 'eie-flood-imagery-prediction/data/xBD_full/'# 
dst_bucket_path='eie-flood-imagery-prediction/data/xBD_flood/'
split_src=('Train_A/' 'Test_A/' 'Hold_A/' 'Tier3_A/' 'Train_B/' 'Test_B/' 'Hold_B/' 'Tier3_B/')
split_dst=('train_A/' 'test_A/' 'hold_A/' 'tier3_A/' 'train_B/' 'test_B/' 'hold_B/' 'tier3_B/')
for ((i=0;i<${#split_src[@]};++i)); do 
    gsutil -m cp -r gs://${bucket_path}${split_src[i]}*.png gs://${dst_bucket_path}${split_dst[i]}
done